In [28]:
import requests
from bs4 import BeautifulSoup
import json
import time

data_list = []

def get_data(url) :
  response = requests.get(url)
  response.encoding = 'utf-8'

  soup = BeautifulSoup(response.text, "html.parser")
  articles = soup.find_all("div", id="io_content")

  if articles:
      for ar in articles:
          h2_tags = ar.find_all('h2')
          if h2_tags:
              title = []
              for h2 in h2_tags:
                  title.append(h2.get_text(strip=True))
          else:
              print("No <h2> found in this <div>")

          p_tags = ar.find_all('p')
          page = []
          if p_tags:
              for p in p_tags:
                  page.append(p.get_text(strip=True))
          else:
              print("No <p> found in this <div>")

          comment = ar.find_all("div", class_="io_center")
          detail = []
          if comment:
              for c in comment:
                  detail.append(c.get_text(strip=True).replace("【註釋】",""))
          else:
              print("No <c> found in this <div>")

          description = ar.find_all("div", class_="io_description")
          descript = []
          if description:
              for d in description:
                  for span in d.find_all("span"):
                      span.decompose()
                  descript.append(d.get_text(strip=True).replace("【道德真經註】",""))
          else:
              print("No <d> found in this <div>")

          Interpretation = ar.find_all("div", class_="io_sanskrit")
          I1 = []
          I2 = []
          if Interpretation:
              for I in Interpretation:
                  text_I = I.get_text(strip=True)
                  if text_I.find("【釋文】") != -1 :
                      I1.append(I.get_text(strip=True).replace("【釋文】",""))
                  elif text_I.find("【解脫】") != -1 :
                      I2.append(I.get_text(strip=True).replace("【解脫】",""))
          else:
              print("No <I> found in this <div>")

          # Append data from this article to data_list
          for i in range(len(h2_tags)):
              data = {}
              data["標題"] = title[i]
              data["文章"] = page[i]
              data["註釋"] = detail[i]
              data["白話文"] = descript[i]
              data["釋文"] = I1[i]
              data["解說"] = I2[i]
              data_list.append(data)
  next_page = soup.find("span", class_="btn2")
  if next_page and next_page.a :
    next_url = f"https://www.ifreesite.com/scriptures/{next_page.a.get('href')}"
    print(f"正在爬取:{next_url}")
    time.sleep(1)
    get_data(next_url)

# Write data_list to JSON Lines file
u = "https://www.ifreesite.com/scriptures/daode-01.htm"
get_data(u)

with open("data.jsonl", "w", encoding="utf-8") as f:
    for data in data_list:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')


正在爬取:https://www.ifreesite.com/scriptures/daode-02.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-03.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-04.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-05.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-06.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-07.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-08.htm
正在爬取:https://www.ifreesite.com/scriptures/daode-09.htm
